In [36]:
import io
import os
import boto3
from azure.storage.blob import BlobServiceClient
import PyPDF2
import requests
from azure.identity import DefaultAzureCredential
from azure.storage.blob import BlobServiceClient

# Set the file path, Azure Blob Storage information, and AWS S3 information
file_path = '' #'file.pdf'

container_name = 'content'
blob_name = 'file.csv'
account_url = 'https://bpf7701sa.blob.core.windows.net/' #'https://myaccount.blob.core.windows.net/'
credential = DefaultAzureCredential() #'mycredential'

bucket_name = 'content'
object_name = 'file.csv'
aws_access_key_id = 'myaccesskey'
aws_secret_access_key = 'mysecretkey'

# Function to read PDF from local file
def read_pdf_local(file_path):
    with open(file_path, 'rb') as f:
        pdf = PdfReader(f)
        text = ""
        for page in pdf.pages:
            text += page.extract_text()
    return text

# Function to read PDF from Azure Blob Storage
def read_pdf_azure(container_name, blob_name, account_url, credential):
    blob_service_client = BlobServiceClient(account_url=account_url, credential=credential)
    blob_client = blob_service_client.get_blob_client(container=container_name, blob=blob_name)
    stream = io.BytesIO()
    blob_client.download_blob().download_to_stream(stream)
    stream.seek(0)
    pdf = PdfReader(stream)
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    return text

# Function to read PDF from AWS S3 Storage
def read_pdf_s3(bucket_name, object_name, aws_access_key_id, aws_secret_access_key):
    s3 = boto3.client('s3', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
    obj = s3.get_object(Bucket=bucket_name, Key=object_name)
    stream = io.BytesIO(obj['Body'].read())
    stream.seek(0)
    pdf = PdfReader(stream)
    text = ""
    for page in pdf.pages:
        text += page.extract_text()
    return text

# Call the appropriate function based on the source of the PDF
if os.path.isfile(file_path):
    text = read_pdf_local(file_path)
elif account_url: #and credential:
    text = read_pdf_azure(container_name, blob_name, account_url, credential)
elif aws_access_key_id and aws_secret_access_key:
    text = read_pdf_s3(bucket_name, object_name, aws_access_key_id, aws_secret_access_key)

# Output the text to another frame
print(text)


PdfReadError: EOF marker not found